In [1]:
import pyspark.sql.functions as f

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7665,application_1589299642358_2161,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure
{
    "conf": {
        "spark.app.name": "datavirus_final"
    }
}

A session has already been started. If you intend to recreate the session with new configurations, please include the -f argument.


In [3]:
# Initialization
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7598,application_1589299642358_2093,pyspark,idle,Link,Link,
7611,application_1589299642358_2106,pyspark,idle,Link,Link,
7617,application_1589299642358_2112,pyspark,idle,Link,Link,
7624,application_1589299642358_2119,pyspark,idle,Link,Link,
7632,application_1589299642358_2126,pyspark,idle,Link,Link,
7633,application_1589299642358_2127,pyspark,busy,Link,Link,
7635,application_1589299642358_2129,pyspark,busy,Link,Link,
7637,application_1589299642358_2131,pyspark,idle,Link,Link,
7640,application_1589299642358_2135,pyspark,idle,Link,Link,
7641,application_1589299642358_2136,pyspark,idle,Link,Link,


In [16]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
sbb.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
|betriebstag|   fahrt_bezeichner|betreiber_id|betreiber_abk|betreiber_name|produkt_id|linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|  bpuic|haltestellen_name|ankunftszeit|an_prognose|an_prognose_status|    abfahrtszeit|ab_prognose|ab_prognose_status|durchfahrt_tf|
+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
| 03.09.2018|80:06____:17010:000|   80:06____|           DB|   DB Regio AG|       Zug|    17010|    

In [18]:
sbb = (
    sbb
    
    # filter out any trips that we are not interested in
    .where(sbb.zusatzfahrt_tf == 'false')
    .where(sbb.faellt_aus_tf == 'false')
    .where(sbb.durchfahrt_tf == 'false')
    .where(sbb.ankunftszeit.isNotNull())
    .where(sbb.an_prognose.isNotNull())
    .where(sbb.an_prognose_status.isin('REAL', 'PROGNOSE'))
    
    # calculate delay
    .withColumn('ankunftszeit_ts', f.unix_timestamp(f.col('ankunftszeit'), "dd.MM.yyyy HH:mm").cast('long'))
    .withColumn('an_prognose_ts', f.unix_timestamp(f.col('an_prognose'), "dd.MM.yyyy HH:mm:ss").cast('long'))
    .withColumn('delay', f.col('an_prognose_ts') - f.col('ankunftszeit_ts'))
    .where(f.col('delay').isNotNull())
    
    # sometimes there are more multiple REAL and PROGNOSE values for the exactly same trip on the same day
    # and time. There is no way which one is the closest to the actual arrival time of the train so we 
    # just drop all duplicates and keep one "randomly".
    .select(['betriebstag', 'fahrt_bezeichner', 'produkt_id', 'linien_id', 'linien_text', 'bpuic', 'ankunftszeit', 'an_prognose_status', 'delay'])
    .dropDuplicates(['betriebstag', 'fahrt_bezeichner', 'bpuic', 'ankunftszeit', 'an_prognose_status'])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('stop_id'))
)
stop_ids = [row['stop_id'] for row in stations.collect()]
stop_ids = spark.sparkContext.broadcast(stop_ids)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
sbb_zurich = sbb.where(sbb.bpuic.isin(stop_ids.value))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
delays = (
    sbb_zurich
    
    # for each trip we only kept one REAL and one PROGNOSE row
    # now we aggregate them into the same row for each trip
    .groupBy(['ankunftszeit', 'fahrt_bezeichner', 'bpuic'])
    .agg(
        f.first(sbb_zurich.produkt_id),
        f.first(sbb_zurich.linien_id),
        f.first(sbb_zurich.linien_text),
        f.first(f.when(sbb_zurich.an_prognose_status == 'REAL', sbb_zurich.delay).otherwise(None)).alias('real_delay'),
        f.first(f.when(sbb_zurich.an_prognose_status == 'PROGNOSE', sbb_zurich.delay).otherwise(None)).alias('prognose_delay'),
    )
    
    # if there is REAL delay we select it, otherwise we take the PROGNOSE delay 
    .withColumn('delay', f.when(f.col('real_delay').isNotNull(), f.col('real_delay')).otherwise(f.col('prognose_delay')))
    .withColumn('is_delayed', f.when(f.col('delay') > 0, 1).otherwise(0))
    
    # create extra columns that contain time without date
    .withColumn('ankunftszeit_minute', delays.ankunftszeit.substr(11, 15))
    .withColumn('ankunftszeit_hour', delays.ankunftszeit.substr(11, 12))
).persist()

An error was encountered:
Session 7665 did not reach idle status in time. Current status is busy.


In [14]:
trip_probability = (
    delays
    
    # group identical trips over all days
    .groupBy(['fahrt_bezeichner', 'bpuic', 'ankunftszeit_minute'])
    
    # calculate delay probability and 1/(mean delay) for each trip
    .agg(
        f.first(delays.produkt_id),
        f.first(delays.linien_id),
        f.first(delays.linien_text),
        f.mean(delays.is_delayed).alias('delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('delay_parameter'),
        f.count(delays.delay).alias('n')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
trip_probability.write.format("orc").save("/user/gflueck/trip_probability.orc", mode='overwrite')

In [ ]:
produkt_probability = (
    delays
    .groupBy(['ankunftszeit_hour', 'produkt_id'])
    .agg(
        f.mean(delays.is_delayed).alias('delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('delay_parameter'),
        f.count(delays.delay).alias('n')
    )

)

In [15]:
produkt_probability.write.format("orc").save("/user/gflueck/produkt_probability.orc", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
probability = spark.read.orc("/user/gflueck/delay_analysis.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
probability.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+-----------------+------------------+--------------------+---+
|    fahrt_bezeichner|  bpuic|ankunftszeit_time| delay_probability|     delay_parameter|  n|
+--------------------+-------+-----------------+------------------+--------------------+---+
|85:3849:64230-050...|8591257|            23:22|0.7142857142857143|0.023632680621201888| 49|
|       85:49:328:000|8505470|                 |               0.0|                null|  0|
|   85:801:81029-3572|8572974|            12:02|0.6690391459074733|0.012789115646258504|281|
|85:820:439107-010...|8589831|            09:54|              0.92|0.010821831869510664| 75|
|85:820:445848-020...|8589794|            22:53|0.7272727272727273| 0.02702702702702703| 11|
+--------------------+-------+-----------------+------------------+--------------------+---+
only showing top 5 rows

In [7]:
probability.select('n').describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|                 n|
+-------+------------------+
|  count|          73021577|
|   mean| 9.149655299830076|
| stddev|28.798668297440553|
|    min|                 0|
|    max|               712|
+-------+------------------+

In [1]:
probability.show(10)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7635,application_1589299642358_2129,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'probability' is not defined
Traceback (most recent call last):
NameError: name 'probability' is not defined

